In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 32.8 MB/s eta 0:00:00


In [ ]:
import pulp as lp
import pandas as pd

# Load data
advertiser_data = pd.DataFrame({
    'Month': ['2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
              '2025-07', '2025-08', '2025-09', '2025-10', '2025-11', '2025-12'],
    'Advertisers': [1024, 996, 1024, 972, 943, 859, 915, 919, 1063, 1499, 1478, 1369],
    'Avg_Budget': [84301, 86866, 86477, 86291, 86199, 84493, 86704, 84993, 87440, 86020, 85232, 86105]
})

# Parameters
initial_agents = 652
annual_salary = 77721
monthly_salary = annual_salary / 12
firing_cost_perc = 0.4
max_adv_per_agent = 10
expected_uplift = 0.135
n_months = len(advertiser_data)

# Initialize PuLP problem
prob = lp.LpProblem("Dynamic_Staffing_Optimization", lp.LpMinimize)

# Decision Variables
months = list(range(n_months))
A = lp.LpVariable.dicts("Available_Agents", months, lowBound=0, cat='Integer')  # Available agents at start of month
N = lp.LpVariable.dicts("New_Hires", months, lowBound=0, cat='Integer')         # New hires in month
F = lp.LpVariable.dicts("Fired", months, lowBound=0, cat='Integer')             # Fired agents in month
S = lp.LpVariable.dicts("Assigned_Advertisers", months, lowBound=0, cat='Integer')  # Advertisers assigned
W = lp.LpVariable.dicts("Waiting_Pool", months, lowBound=0, cat='Integer')       # Advertisers waiting

In [ ]:
# Initial Conditions
prob += A[0] == initial_agents  # Starting agents
W_prev = 0  # Initial waiting pool

# Constraints
for m in months:
    # Agent availability (T_m = A[m] + N_prev - F_prev)
    if m == 0:
        T = A[m]  # No hires/fires before month 1
    else:
        T = A[m] + N[m-1] - F[m-1]

    # 1. Advertiser Assignment: S[m] <= T * max_adv_per_agent
    prob += S[m] <= T * max_adv_per_agent

    # 2. Waiting Pool: W[m] = W_prev + new advertisers - assigned
    prob += W[m] == W_prev + advertiser_data['Advertisers'][m] - S[m]
    prob += W[m] >= 0

    # 3. Hiring/Firing Constraints
    prob += N[m] <= 0.2 * A[m]  # Max 20% hiring
    prob += F[m] <= 0.25 * A[m]  # Max 25% firing

    # 4. Required Agents: T * 10 >= S[m] + S_prev (eliminates division)
    if m == 0:
        prob += T * 10 >= S[m]  # Equivalent to T >= S[m]/10
    else:
        prob += T * 10 >= S[m] + S[m-1]  # Support for current + prior month

    # Update next month's available agents
    if m < n_months - 1:
        prob += A[m+1] == A[m] + N[m] - F[m]

    W_prev = W[m]  # Update waiting pool for next iteration

# Objective Function minimized
total_cost = lp.lpSum(
    [A[m] * monthly_salary + F[m] * firing_cost_perc * annual_salary
     - S[m] * advertiser_data['Avg_Budget'][m] * expected_uplift
     for m in months]
)
prob += total_cost

# waiting pool 60+, waiting cost (annual budget/12 *0.135) to solve the vacancy in first month, one agent service one advertise for full 2 months.

# Solve
prob.solve()

# Results
results = []
for m in months:
    results.append({
        'Month': advertiser_data['Month'][m],
        'Available Agents': A[m].varValue,
        'New Hires': N[m].varValue,
        'Fired': F[m].varValue,
        'Assigned Advertisers': S[m].varValue,
        'Total Cost ($)': round(A[m].varValue * monthly_salary + F[m].varValue * firing_cost_perc * annual_salary, 2),
        'Revenue Uplift ($)': round(S[m].varValue * advertiser_data['Avg_Budget'][m] * expected_uplift, 2)
    })

In [ ]:
results_df = pd.DataFrame(results)
total_annual_cost = results_df['Total Cost ($)'].sum()
total_uplift = results_df['Revenue Uplift ($)'].sum()
net_profit = total_uplift - total_annual_cost

print(results_df.to_string(index=False))
print(f"\nTotal Annual Cost: ${total_annual_cost:,.2f}")
print(f"Total Revenue Uplift: ${total_uplift:,.2f}")
print(f"Net Profit: ${net_profit:,.2f}")

  Month  Available Agents  New Hires  Fired  Assigned Advertisers  Total Cost ($)  Revenue Uplift ($)
2025-01             652.0        0.0  163.0                   0.0      9290250.20                0.00
2025-02             489.0        0.0  122.0                2020.0      6959915.55         23688358.20
2025-03             367.0        0.0   91.0                 430.0      5206011.65          5019989.85
2025-04             276.0        0.0   41.0                1420.0      3062207.40         16541984.70
2025-05             235.0        0.0    0.0                 520.0      1522036.25          6051169.80
2025-06             235.0        0.0    0.0                   1.0      1522036.25            11406.56
2025-07             235.0        0.0    0.0                2342.0      1522036.25         27413203.68
2025-08             235.0        0.0    0.0                   8.0      1522036.25            91792.44
2025-09             235.0       20.0    0.0                1974.0      1522036.25 